这类哈希函数有以下特点

消息填充方式都比较类似，首先在消息后面添加一个1，然后填充若干个0，直至总长度与 448 同余，最后在其后附上64位的消息长度（填充前）。
每一块得到的链接变量都会被作为下一次执行hash函数的初始向量IV。在最后一块的时候，才会将其对应的链接变量转换为hash值。
一般攻击时应满足如下条件

我们已知 key 的长度，如果不知道的话，需要爆破出来
我们可以控制 message 的消息。
我们已经知道了包含 key 的一个消息的hash值。
这样我们就可以得到一对(messge,x)满足x=H(key ∥ message)虽然我们并不清楚key的内容。

攻击原理¶
这里不妨假设我们我们知道了 hash(key+s) 的 hash 值，其中 s 是已知的，那么其本身在计算的时候，必然会进行填充。那么我们首先可以得到 key+s 扩展后的字符串 now，即

now=key|s|padding

那么如果我们在 now 的后面再次附加上一部分信息extra，即

key|s|padding|extra

这样再去计算hash值的时候，

会对 extra 进行填充直到满足条件。
先计算 now 对应的链接变量 IV1，而我们已经知道这部分的 hash 值，并且链接变量产生 hash 值的算法是可逆的，所以我们可以得到链接变量。
下面会根据得到的链接变量 IV1，对 extra 部分进行哈希算法，并返回hash值。
那么既然我们已经知道了第一部分的 hash 值，并且，我们还知道 extra 的值，那么我们便可以得到最后的hash值。

而之前我们也说了我们可以控制 message 的值。那么其实 s，padding，extra 我们都是可以控制的。所以我们自然可以找到对应的(message,x)满足x=hash(key|message)。

In [21]:
import binascii
from gmssl import sm3


# 读取HMAC key文件
def read_hmac_key(file_path):
    with open(file_path, 'rb') as f:
        hmac_key = f.read().strip()
    return hmac_key


# 生成token
def generate_token(hmac_key, counter):
    # 如果HMAC_KEY长度不足32字节，则在末尾补0，超过64字节则截断
    if len(hmac_key) < 32:
        hmac_key = hmac_key.ljust(32, b'\x00')
    elif len(hmac_key) > 32:
        hmac_key = hmac_key[:32]

    # 将计数器转换为字节表示
    counter_bytes = counter.to_bytes((counter.bit_length() + 7) // 8, 'big')
    print("hmac_key:", binascii.hexlify(hmac_key))
    print("counter_bytes:", binascii.hexlify(counter_bytes))
    print(hmac_key + counter_bytes)
    tobe_hashed = bytearray(hmac_key + counter_bytes)
    # print(tobe_hashed)
    print("tobe_hashed:", binascii.hexlify(tobe_hashed))

    # 使用SM3算法计算哈希值
    sm3_hash = sm3.sm3_hash(tobe_hashed)

    # 将SM3的哈希值转换为十六进制字符串作为token
    token = sm3_hash

    return token


current_counter = 0


def verify_token(hmac_key, counter, token):
    # 生成token
    generated_token = generate_token(hmac_key, counter)
    global current_counter
    # 比较生成的token和输入的token是否相同
    if generated_token == token:
        if counter & 0xFFFFFFFF > current_counter:
            current_counter = counter & 0xFFFFFFFF
            print("current_counter: ", hex(current_counter))
            return "Success"
        else:
            return "Error: counter must be increasing"
    else:
        return "Error: token not match"



In [23]:
# 假设HMAC key文件路径
# hmac_key_file = 'hmac_key.txt'
# 假设计数器值
counter = 0x12345678

# 读取HMAC key
# hmac_key = read_hmac_key(hmac_key_file)
hmac_key = b""

# 生成token
token = generate_token(hmac_key, counter)
print("Generated token:", token)
# print(verify_token(hmac_key, counter, token))

hmac_key: b'0000000000000000000000000000000000000000000000000000000000000000'
counter_bytes: b'12345678'
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x124Vx'
tobe_hashed: b'000000000000000000000000000000000000000000000000000000000000000012345678'
Generated token: 6898b1d01eaa4a9123c56482dcf4b3249ab5b9ef40c4a91f3644989c730f6ac3


counter: 0x56fa775a
token: 0x571c0dcca05d5afc59c8de7a373adc25529e5fc240355765150317d7ca385b48

In [75]:
counter = 0x56fa775a
# counter = 0x12345678

hash1 = 0x571c0dcca05d5afc59c8de7a373adc25529e5fc240355765150317d7ca385b48

ori='000000000000000000000000000000000000000000000000000000000000000056fa775a80000000000000000000000000000000000000000000000000000120'
ori='000000000000000000000000000000000000000000000000000000000000000056fa775a80000000000000000000000000000000000000000000000000000120000000000000000000000000000000000000000000000000000000000000000056fa775a80000000000000000000000000000000000000000000000000000120'

print(f"ori={ori}")
# now=key|s|padding
# Hash(now)= IV1

hmac_key = b""

# 如果HMAC_KEY长度不足32字节，则在末尾补0，超过64字节则截断
if len(hmac_key) < 32:
    hmac_key = hmac_key.ljust(32, b'\x00')
elif len(hmac_key) > 32:
    hmac_key = hmac_key[:32]

# 将计数器转换为字节表示
counter_bytes = counter.to_bytes((counter.bit_length() + 7) // 8, 'big')
print("hmac_key:", binascii.hexlify(hmac_key))
print("counter_bytes:", binascii.hexlify(counter_bytes))
print(hmac_key + counter_bytes)
tobe_hashed = bytearray(hmac_key + counter_bytes)
# print(tobe_hashed)
print("tobe_hashed:", binascii.hexlify(tobe_hashed))
counter_bytes = counter.to_bytes((counter.bit_length() + 7) // 8, 'big')

msg=bytearray(hmac_key+counter_bytes)

# print(msg)
# len1 = len(msg)
# reserve1 = len1 % 64
# msg.append(0x80)
# reserve1 = reserve1 + 1
# # 56-64, add 64 byte
# range_end = 56
# if reserve1 > range_end:
#     range_end = range_end + 64

# for i in range(reserve1, range_end):
#     msg.append(0x00)
# print(f"padding_msg={binascii.hexlify(msg)}")
# bit_length = (len1) * 8
# bit_length_str = [bit_length % 0x100]
# for i in range(7):
#     bit_length = int(bit_length / 0x100)
#     bit_length_str.append(bit_length % 0x100)
# for i in range(8):
#     msg.append(bit_length_str[7-i])
# print(f"padding_num_msg={binascii.hexlify(msg)}")


msg = bytearray(bytes.fromhex(ori))

print(f"padding_num_msg={binascii.hexlify(msg)}")

group_count = round(len(msg) / 64)
print(f"grout_count={group_count}")
B = []
for i in range(0, group_count):
    B.append(msg[i*64:(i+1)*64])
print(f"B={B}")
# 连接变量V
V = []
V.append(sm3.IV)
for i in range(0, group_count):
    V.append(sm3.sm3_cf(V[i], B[i]))
print(f"V={V}")

y = V[i+1]
result = ""
for i in y:
    result = '%s%08x' % (result, i)
print(f"token={result}")




ori=000000000000000000000000000000000000000000000000000000000000000056fa775a80000000000000000000000000000000000000000000000000000120000000000000000000000000000000000000000000000000000000000000000056fa775a80000000000000000000000000000000000000000000000000000120
hmac_key: b'0000000000000000000000000000000000000000000000000000000000000000'
counter_bytes: b'56fa775a'
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00V\xfawZ'
tobe_hashed: b'000000000000000000000000000000000000000000000000000000000000000056fa775a'
padding_num_msg=b'000000000000000000000000000000000000000000000000000000000000000056fa775a80000000000000000000000000000000000000000000000000000120000000000000000000000000000000000000000000000000000000000000000056fa775a80000000000000000000000000000000000000000000000000000120'
grout_count=2
B=[bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [85]:
# 通过加密的token找到链接向量
token="571c0dcca05d5afc59c8de7a373adc25529e5fc240355765150317d7ca385b48"
IV1=[]
for i in range (0,8):
    tmp=token[i*8:(i+1)*8]
    # print(int(tmp,16))
    IV1.append(int(tmp,16))
    
print(f"IV1={IV1}")

# IV1=[1461456332, 2690472700, 1506336378, 926604325, 1386110914, 1077237605, 352524247, 3392691016]



ori='000000000000000000000000000000000000000000000000000000000000000056fa775a80000000000000000000000000000000000000000000000000000120'

new_ori='000000000000000000000000000000000000000000000000000000000000000056fa775a80000000000000000000000000000000000000000000000000000120000000000000000000000000000000000000000000000000000000000000000056fa775b80000000000000000000000000000000000000000000000000000320'

#计算第二个block
B=bytearray(bytes.fromhex("000000000000000000000000000000000000000000000000000000000000000056fa775b80000000000000000000000000000000000000000000000000000320"))

res=sm3.sm3_cf(IV1, B)


y = res
result = ""
for i in y:
    result = '%s%08x' % (result, i)
print(f"token={result}")


#找到一个counter，使填充后等于new_ori
# 56fa775a80000000000000000000000000000000000000000000000000000120000000000000000000000000000000000000000000000000000000000000000056fa775b
# af18d3d5c48e246f87c19553a0fd8dd97981be1e419d0b182d9bff11a640f1a3


IV1=[1461456332, 2690472700, 1506336378, 926604325, 1386110914, 1077237605, 352524247, 3392691016]
token=af18d3d5c48e246f87c19553a0fd8dd97981be1e419d0b182d9bff11a640f1a3
